In [1]:
# 데이터 분석
import pandas as pd
import numpy as np

# 통계
import pingouin as pg
from scipy.stats import chisquare

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# 컬럼 출력 생략 해제
pd.set_option('display.max_columns', None)

# 시각화 한글 표시
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# 머신러닝
from sklearn import preprocessing

In [2]:
# 데이터 로드
df_raw = pd.read_csv("https://raw.githubusercontent.com/JounKK/AIS8_Final_HRA/main/dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv", index_col="EmployeeNumber")
df_raw.shape

(1470, 34)

In [3]:
# 불필요한 컬럼 제거
df_raw = df_raw.drop(columns=['Over18', 'EmployeeCount', 'StandardHours','DailyRate', 'HourlyRate', 'MonthlyRate'])
df_raw.shape

(1470, 28)

**언더샘플링**

In [15]:
# 언더샘플링
df_raw['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [17]:
df_yes = df_raw[df_raw['Attrition']=='Yes']
df_no = df_raw[df_raw['Attrition']=='No'].sample(237)

df_yes.shape, df_no.shape

((237, 28), (237, 28))

In [18]:
df_under = pd.concat([df_yes, df_no])

In [19]:
df_under['Attrition'].value_counts()

Yes    237
No     237
Name: Attrition, dtype: int64

**인코딩**

In [51]:
# Object 컬럼들
categorical_column = ['Attrition', 'BusinessTravel', 'Department', 'EducationField',
                      'Gender', 'JobRole', 'MaritalStatus', 'OverTime']

# 깊은 복사 해주기
df_encoded = df_raw.copy(deep=True)

# Scikit-learn 사용해서 라벨 인코딩하기
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        df_encoded[col] = lab_enc.fit_transform(df_raw[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)

df_encoded.head()

Feature Attrition
mapping {'No': 0, 'Yes': 1}
Feature BusinessTravel
mapping {'Non-Travel': 0, 'Travel_Frequently': 1, 'Travel_Rarely': 2}
Feature Department
mapping {'Human Resources': 0, 'Research & Development': 1, 'Sales': 2}
Feature EducationField
mapping {'Human Resources': 0, 'Life Sciences': 1, 'Marketing': 2, 'Medical': 3, 'Other': 4, 'Technical Degree': 5}
Feature Gender
mapping {'Female': 0, 'Male': 1}
Feature JobRole
mapping {'Healthcare Representative': 0, 'Human Resources': 1, 'Laboratory Technician': 2, 'Manager': 3, 'Manufacturing Director': 4, 'Research Director': 5, 'Research Scientist': 6, 'Sales Executive': 7, 'Sales Representative': 8}
Feature MaritalStatus
mapping {'Divorced': 0, 'Married': 1, 'Single': 2}
Feature OverTime
mapping {'No': 0, 'Yes': 1}


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeNumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,41,1,2,2,1,2,1,2,0,3,2,7,4,2,5993,8,1,11,3,1,0,8,0,1,6,4,0,5
2,49,0,1,1,8,1,1,3,1,2,2,6,2,1,5130,1,0,23,4,4,1,10,3,3,10,7,1,7
4,37,1,2,1,2,2,4,4,1,2,1,2,3,2,2090,6,1,15,3,2,0,7,3,3,0,0,0,0
5,33,0,1,1,3,4,1,4,0,3,1,6,3,1,2909,1,1,11,3,3,0,8,3,3,8,7,3,0
7,27,0,2,1,2,1,3,1,1,3,1,2,2,1,3468,9,0,12,3,4,1,6,3,3,2,2,2,2


**오버샘플링**

In [26]:
# df_attr = df_raw['Attrition'].to_frame()
# df_attr.head(1)

,Attrition
EmployeeNumber,
1,Yes


In [27]:
# df_num = df_raw[number_cols]
# df_num.head(1)

,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeNumber,,,,,,,,,,,
1,41,1,5993,8,11,8,0,6,4,0,5


In [37]:
# df_over = pd.concat([df_attr, df_num], axis=1)
# df_over

,Attrition,Age,DistanceFromHome,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeNumber,,,,,,,,,,,,
1,Yes,41,1,5993,8,11,8,0,6,4,0,5
2,No,49,8,5130,1,23,10,3,10,7,1,7
4,Yes,37,2,2090,6,15,7,3,0,0,0,0
5,No,33,3,2909,1,11,8,3,8,7,3,0
7,No,27,2,3468,9,12,6,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2061,No,36,23,2571,4,17,17,3,5,2,0,3
2062,No,39,6,9991,4,15,9,5,7,7,1,7
2064,No,27,4,6142,1,20,6,0,6,2,0,3


In [53]:
df_over = df_encoded.copy()

In [54]:
label_name = "Attrition"

X = df_over.drop(label_name, axis=1)
y = df_over[label_name]
X.shape, y.shape

((1470, 27), (1470,))

In [55]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_over, y_over = sm.fit_resample(X, y)

In [56]:
X_over.shape, y_over.shape

((2466, 27), (2466,))

In [57]:
# y의 value_counts
y.value_counts(1)

0    0.838776
1    0.161224
Name: Attrition, dtype: float64

In [58]:
# y_resample의 value_counts
y_over.value_counts(1)

1    0.5
0    0.5
Name: Attrition, dtype: float64

In [61]:
df_over = pd.concat([X_over, y_over], axis=1)
df_over

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,41,2,2,1,2,1,2,0,3,2,7,4,2,5993,8,1,11,3,1,0,8,0,1,6,4,0,5,1
1,49,1,1,8,1,1,3,1,2,2,6,2,1,5130,1,0,23,4,4,1,10,3,3,10,7,1,7,0
2,37,2,1,2,2,4,4,1,2,1,2,3,2,2090,6,1,15,3,2,0,7,3,3,0,0,0,0,1
3,33,1,1,3,4,1,4,0,3,1,6,3,1,2909,1,1,11,3,3,0,8,3,3,8,7,3,0,0
4,27,2,1,2,1,3,1,1,3,1,2,2,1,3468,9,0,12,3,4,1,6,3,3,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,24,2,1,12,1,4,4,0,3,1,6,4,2,2690,1,0,17,3,1,0,3,2,2,3,2,2,2,1
2462,33,0,2,18,3,2,1,0,3,2,7,3,2,5310,7,0,13,3,2,0,8,3,2,4,2,0,3,1
2463,47,1,1,1,4,3,3,1,3,1,6,3,1,4735,3,1,14,3,3,0,4,3,3,0,0,0,0,1
2464,23,2,1,5,1,2,2,0,3,1,7,1,2,2673,2,0,12,3,2,0,2,3,3,2,0,0,1,1


In [ ]:
# 원핫 인코딩 했던걸 다시 문자형으로 바꿔서 해보기. 차이가 있나없나...

**변수 유형별 모음**

In [4]:
# 수치형 변수 모음
number_cols = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
               'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
               'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
               'YearsWithCurrManager']
number_cols

['Age',
 'DistanceFromHome',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [7]:
# 순서형 변수 모음
ordinal_cols = ['Education', 'EnvironmentSatisfaction', 'JobInvolvement',
                'JobLevel', 'JobSatisfaction', 'PerformanceRating',
                'RelationshipSatisfaction', 'StockOptionLevel', 'WorkLifeBalance']
ordinal_cols

['Education',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'WorkLifeBalance']

In [6]:
# 범주형 변수 모음
cat_cols = ['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
cat_cols

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [8]:
# 수치형 + 범주형 변수 모음
cat_ord_list = ordinal_cols + cat_cols
cat_ord_list

['Education',
 'EnvironmentSatisfaction',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'WorkLifeBalance',
 'Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

## 수치형. df_raw. t-test

In [14]:
# Attrition에 따른 수치형 변수간 t-test
# Attrition은 각각의 수치형 변수에 대해 유의미한 차이가 있는가?


try:
    for col in number_cols:
        Attrition_yes = df_raw[df_raw['Attrition'] == 'Yes'][col]
        Attrition_no = df_raw[df_raw['Attrition'] == 'No'][col]
        display(col, pg.ttest(Attrition_yes, Attrition_no))
except:
    pass

'Age'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.828012,316.931112,two-sided,1.379760e-08,"[-5.29, -2.62]",0.438225,1.181e+06,1.0


'DistanceFromHome'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.888183,322.724279,two-sided,0.004137,"[0.55, 2.89]",0.212401,4.71,0.849148


'MonthlyIncome'

...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-6.847079,366.568825,two-sided,3.187390e-11,"[-2.04, -1.13]",0.442011,5.811e+08,1.0


'YearsSinceLastPromotion'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.287927,338.491595,two-sided,0.198651,"[-0.73, 0.15]",0.089777,0.179,0.244156


'YearsWithCurrManager'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-6.633399,365.098101,two-sided,1.185022e-10,"[-1.96, -1.07]",0.429743,1.464e+08,1.0


## 수치형. df_under. t-test

In [43]:
# Attrition에 따른 수치형 변수간 t-test
# Attrition은 각각의 수치형 변수에 대해 유의미한 차이가 있는가?


try:
    for col in number_cols:
        Attrition_yes = df_under[df_under['Attrition'] == 'Yes'][col]
        Attrition_no = df_under[df_under['Attrition'] == 'No'][col]
        display(col, pg.ttest(Attrition_yes, Attrition_no))
except:
    pass

'Age'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.884612,472,two-sided,0.000001,"[-5.96, -2.54]",0.448715,8592.892,0.99822


'DistanceFromHome'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.100725,472,two-sided,0.036195,"[0.1, 3.03]",0.192979,0.862,0.554306


'MonthlyIncome'

...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.601423,472,two-sided,0.000005,"[-1.93, -0.78]",0.422701,2454.553,0.995757


'YearsSinceLastPromotion'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.334286,472,two-sided,0.738313,"[-0.64, 0.45]",0.030708,0.108,0.062844


'YearsWithCurrManager'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.160077,472,two-sided,0.000038,"[-1.91, -0.69]",0.382157,400.002,0.985797


## 수치형. df_over. t-test

In [63]:
# Attrition에 따른 수치형 변수간 t-test
# Attrition은 각각의 수치형 변수에 대해 유의미한 차이가 있는가?


try:
    for col in number_cols:
        Attrition_yes = df_over[df_over['Attrition'] == 1][col]
        Attrition_no = df_over[df_over['Attrition'] == 0][col]
        display(col, pg.ttest(Attrition_yes, Attrition_no))
except:
    pass

'Age'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-12.991969,2464,two-sided,2.230392e-37,"[-5.25, -3.88]",0.523249,8.596e+33,1.0


'DistanceFromHome'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.036517,2464,two-sided,5.083946e-07,"[0.95, 2.15]",0.202845,1.272e+04,0.998946


'MonthlyIncome'

...

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-14.133474,2464,two-sided,1.145011e-43,"[-2.1, -1.59]",0.569223,1.464e+40,1.0


'YearsSinceLastPromotion'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.431943,2464,two-sided,0.00001,"[-0.78, -0.3]",0.178496,758.141,0.993249


'YearsWithCurrManager'

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-13.858219,2464,two-sided,4.135209e-42,"[-2.04, -1.53]",0.558137,4.185e+38,1.0


## 범주형. df_raw. chi-square

In [64]:
# Attrition에 따른 범주형(순서형) 변수간 카이제곱 검정
# 귀무가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 없다.
# 대립가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 있다.

try:
    for col in cat_ord_list:
        expected, observed, stats = pg.chi2_independence(x='Attrition', y=col, data=df_raw)
        display(df_raw.pivot_table(index=col, columns='Attrition', aggfunc='size')) # 분할표
        display(stats)
except:
    pass

Attrition,No,Yes
Education,,
1,139,31
2,238,44
3,473,99
4,340,58
5,43,5


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,3.073961,4.0,0.545525,0.045729,0.249710
1,cressie-read,0.666667,3.112460,4.0,0.539185,0.046014,0.252603
2,log-likelihood,0.000000,3.199999,4.0,0.524931,0.046657,0.259195
3,freeman-tukey,-0.500000,3.276024,4.0,0.512740,0.047208,0.264935
4,mod-log-likelihood,-1.000000,3.362199,4.0,0.499138,0.047825,0.271459
5,neyman,-2.000000,3.570535,4.0,0.467235,0.049284,0.287287


Attrition,No,Yes
EnvironmentSatisfaction,,
1,212,72
2,244,43
3,391,62
4,386,60


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,22.503881,3.0,0.000051,0.123729,0.986916
1,cressie-read,0.666667,21.798382,3.0,0.000072,0.121774,0.984270
2,log-likelihood,0.000000,20.575349,3.0,0.000129,0.118308,0.978454
3,freeman-tukey,-0.500000,19.802549,3.0,0.000187,0.116065,0.973799
4,mod-log-likelihood,-1.000000,19.137295,3.0,0.000256,0.114099,0.969058
5,neyman,-2.000000,18.081705,3.0,0.000423,0.110908,0.959879


Attrition,No,Yes
JobInvolvement,,
1,55,28
2,304,71
3,743,125
4,131,13


...

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,86.190254,8.0,2.752482e-15,0.242142,1.0
1,cressie-read,0.666667,85.838001,8.0,3.243442e-15,0.241647,1.0
2,log-likelihood,0.000000,88.908721,8.0,7.743392e-16,0.245931,1.0
3,freeman-tukey,-0.500000,95.201061,8.0,4.070829e-17,0.254485,1.0
4,mod-log-likelihood,-1.000000,106.301702,8.0,2.187725e-19,0.268913,1.0
5,neyman,-2.000000,155.087408,8.0,1.700565e-29,0.324810,1.0


Attrition,No,Yes
MaritalStatus,,
Divorced,294,33
Married,589,84
Single,350,120


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,46.163677,2.0,9.455511e-11,0.177211,0.999996
1,cressie-read,0.666667,45.300140,2.0,1.456128e-10,0.175546,0.999995
2,log-likelihood,0.000000,43.999698,2.0,2.789890e-10,0.173008,0.999992
3,freeman-tukey,-0.500000,43.373275,2.0,3.816041e-10,0.171772,0.999990
4,mod-log-likelihood,-1.000000,43.029524,2.0,4.531663e-10,0.171090,0.999988
5,neyman,-2.000000,43.161501,2.0,4.242278e-10,0.171352,0.999989


Attrition,No,Yes
OverTime,,
No,944,110
Yes,289,127


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,87.564294,1.0,8.158424e-21,0.244065,1.0
1,cressie-read,0.666667,84.576205,1.0,3.696877e-20,0.239864,1.0
2,log-likelihood,0.000000,80.079543,1.0,3.596366e-19,0.233401,1.0
3,freeman-tukey,-0.500000,77.830595,1.0,1.122677e-18,0.230100,1.0
4,mod-log-likelihood,-1.000000,76.421211,1.0,2.291765e-18,0.228007,1.0
5,neyman,-2.000000,75.828553,1.0,3.093947e-18,0.227121,1.0


## 범주형. df_under. chi-sqaure

In [65]:
# Attrition에 따른 범주형(순서형) 변수간 카이제곱 검정
# 귀무가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 없다.
# 대립가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 있다.

try:
    for col in cat_ord_list:
        expected, observed, stats = pg.chi2_independence(x='Attrition', y=col, data=df_under)
        display(df_under.pivot_table(index=col, columns='Attrition', aggfunc='size')) # 분할표
        display(stats)
except:
    pass

Attrition,No,Yes
Education,,
1,22,31
2,51,44
3,82,99
4,75,58
5,7,5


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,6.147042,4.0,0.188433,0.113879,0.480669
1,cressie-read,0.666667,6.150979,4.0,0.188153,0.113916,0.480950
2,log-likelihood,0.000000,6.164813,4.0,0.187173,0.114044,0.481940
3,freeman-tukey,-0.500000,6.180453,4.0,0.186071,0.114188,0.483057
4,mod-log-likelihood,-1.000000,6.200663,4.0,0.184655,0.114375,0.484499
5,neyman,-2.000000,6.255061,4.0,0.180893,0.114875,0.488373


Attrition,No,Yes
EnvironmentSatisfaction,,
1,41,72
2,49,43
3,68,62
4,79,60


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,11.769775,3.0,0.008215,0.157578,0.832399
1,cressie-read,0.666667,11.795847,3.0,0.008116,0.157752,0.833302
2,log-likelihood,0.000000,11.888321,3.0,0.007776,0.158369,0.836469
3,freeman-tukey,-0.500000,11.994114,3.0,0.007403,0.159072,0.840031
4,mod-log-likelihood,-1.000000,12.132518,3.0,0.006943,0.159988,0.844592
5,neyman,-2.000000,12.513623,3.0,0.005816,0.162481,0.856588


Attrition,No,Yes
JobInvolvement,,
1,7,28
2,57,71
3,152,125
4,21,13


...

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,45.984350,8.0,2.393332e-07,0.311470,0.999783
1,cressie-read,0.666667,46.482040,8.0,1.924540e-07,0.313151,0.999809
2,log-likelihood,0.000000,48.397737,8.0,8.291972e-08,0.319539,0.999885
3,freeman-tukey,-0.500000,50.847360,8.0,2.807848e-08,0.327525,0.999941
4,mod-log-likelihood,-1.000000,54.482439,8.0,5.564892e-09,0.339031,0.999978
5,neyman,-2.000000,67.500350,8.0,1.542777e-11,0.377367,0.999999


Attrition,No,Yes
MaritalStatus,,
Divorced,48,33
Married,116,84
Single,73,120


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,19.343374,2.0,0.000063,0.202012,0.982276
1,cressie-read,0.666667,19.377356,2.0,0.000062,0.202189,0.982441
2,log-likelihood,0.000000,19.497444,2.0,0.000058,0.202815,0.983011
3,freeman-tukey,-0.500000,19.634189,2.0,0.000055,0.203525,0.983640
4,mod-log-likelihood,-1.000000,19.812225,2.0,0.000050,0.204445,0.984425
5,neyman,-2.000000,20.298088,2.0,0.000039,0.206937,0.986390


Attrition,No,Yes
OverTime,,
No,184,110
Yes,53,127


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,47.731406,1.0,4.887983e-12,0.317331,1.0
1,cressie-read,0.666667,47.962083,1.0,4.345419e-12,0.318097,1.0
2,log-likelihood,0.000000,48.792612,1.0,2.845096e-12,0.320840,1.0
3,freeman-tukey,-0.500000,49.761107,1.0,1.736523e-12,0.324008,1.0
4,mod-log-likelihood,-1.000000,51.053854,1.0,8.986629e-13,0.328190,1.0
5,neyman,-2.000000,54.750269,1.0,1.368605e-13,0.339863,1.0


## 범주형. df_over. chi-square

In [66]:
# Attrition에 따른 범주형(순서형) 변수간 카이제곱 검정
# 귀무가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 없다.
# 대립가설 : 범주형(순서형) 변수에 따라 퇴사 비율에 차이가 있다.

try:
    for col in cat_ord_list:
        expected, observed, stats = pg.chi2_independence(x='Attrition', y=col, data=df_over)
        display(df_over.pivot_table(index=col, columns='Attrition', aggfunc='size')) # 분할표
        display(stats)
except:
    pass

Attrition,0,1
Education,,
1,139,181
2,238,334
3,473,567
4,340,146
5,43,5


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,137.644205,4.0,9.014335e-29,0.236256,1.0
1,cressie-read,0.666667,139.001164,4.0,4.618208e-29,0.237417,1.0
2,log-likelihood,0.000000,144.329441,4.0,3.338580e-30,0.241925,1.0
3,freeman-tukey,-0.500000,151.338604,4.0,1.051626e-31,0.247730,1.0
4,mod-log-likelihood,-1.000000,162.092617,4.0,5.201093e-34,0.256381,1.0
5,neyman,-2.000000,203.468219,4.0,6.747645e-43,0.287245,1.0


Attrition,0,1
EnvironmentSatisfaction,,
1,212,401
2,244,368
3,391,352
4,386,112


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,236.198740,3.0,6.316860e-51,0.309487,1.0
1,cressie-read,0.666667,238.290635,3.0,2.229210e-51,0.310854,1.0
2,log-likelihood,0.000000,246.046808,3.0,4.686169e-53,0.315873,1.0
3,freeman-tukey,-0.500000,255.441340,3.0,4.354052e-55,0.321847,1.0
4,mod-log-likelihood,-1.000000,268.499248,3.0,6.518523e-58,0.329970,1.0
5,neyman,-2.000000,308.850993,3.0,1.208017e-66,0.353898,1.0


Attrition,0,1
JobInvolvement,,
1,55,187
2,304,551
3,743,477
4,131,18


...

,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,65.614446,8.0,3.647893e-11,0.163118,0.999999
1,cressie-read,0.666667,66.231709,8.0,2.753116e-11,0.163884,0.999999
2,log-likelihood,0.000000,68.531488,8.0,9.628320e-12,0.166705,1.000000
3,freeman-tukey,-0.500000,71.333933,8.0,2.664971e-12,0.170079,1.000000
4,mod-log-likelihood,-1.000000,75.253293,8.0,4.388864e-13,0.174689,1.000000
5,neyman,-2.000000,87.500268,8.0,1.494342e-15,0.188368,1.000000


Attrition,0,1
MaritalStatus,,
0,294,207
1,589,646
2,350,380


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,18.971430,2.0,0.000076,0.087711,0.980375
1,cressie-read,0.666667,18.988722,2.0,0.000075,0.087751,0.980467
2,log-likelihood,0.000000,19.049573,2.0,0.000073,0.087891,0.980789
3,freeman-tukey,-0.500000,19.118502,2.0,0.000071,0.088050,0.981148
4,mod-log-likelihood,-1.000000,19.207754,2.0,0.000067,0.088255,0.981604
5,neyman,-2.000000,19.448876,2.0,0.000060,0.088808,0.982783


Attrition,0,1
OverTime,,
0,944,804
1,289,429


,test,lambda,chi2,dof,pval,cramer,power
0,pearson,1.000000,37.962674,1.0,7.211109e-10,0.124074,0.999987
1,cressie-read,0.666667,38.002989,1.0,7.063635e-10,0.124140,0.999987
2,log-likelihood,0.000000,38.145012,1.0,6.567743e-10,0.124372,0.999988
3,freeman-tukey,-0.500000,38.306106,1.0,6.047313e-10,0.124634,0.999988
4,mod-log-likelihood,-1.000000,38.514995,1.0,5.433461e-10,0.124974,0.999989
5,neyman,-2.000000,39.080805,1.0,4.066222e-10,0.125888,0.999991
